In [47]:
import numpy as np
import xtrack as xt         
import xpart as xp
import xdeps as xd   
import yaml
import os
import inspect
import pickle
import scipy

In [3]:
!ls


checks.ipynb  knobs_db	lines_ref  match_phase.py  phaseAdvances.ipynb


In [11]:
with open('../HL_IR7_phase_advance/knob.pkl', 'rb') as f:
    knobs1 = pickle.load(f)

In [12]:
knobs == knobs1

True

In [48]:
beam          = 2   #run_dict['beam']
plane         = "V" #run_dict['plane']
TCCS_name     = 'tccs.5r3.b2'
TCCP_name     = 'tccp.4l3.b2'
TARGET_name   = 'target.4l3.b2'
TCLA_name     = 'tcla.a5l3.b2'
TCP_name      = f"tcp.{'c' if plane=='H' else 'd'}6{'l' if beam=='1' else 'r'}7.b{beam}"

def load_line(line_name, coll_file, particle_ref):
    if coll_file.endswith('.yaml'):
        with open(coll_file, 'r') as stream:
            coll_dict = yaml.safe_load(stream)['collimators'][f'b2']

    # Load from json
    line = xt.Line.from_json(line_name)
    line.particle_ref = particle_ref

    end_s = line.get_length()

    TCCS_loc = end_s - 6773.7 #6775
    TCCP_loc = end_s - 6653.3 #6655
    TARGET_loc = end_s - (6653.3 + coll_dict[TCCP_name]["length"]/2 + coll_dict[TARGET_name]["length"]/2)

    line.insert_element(at_s=TCCS_loc, element=xt.Marker(), name='tccs.5r3.b2')
    line.insert_element(at_s=TCCS_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='tccs.5r3.b2_aper')
    line.insert_element(at_s=TCCP_loc, element=xt.Marker(), name='tccp.4l3.b2')
    line.insert_element(at_s=TCCP_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='tccp.4l3.b2_aper')
    line.insert_element(at_s=TARGET_loc, element=xt.Marker(), name='target.4l3.b2')
    line.insert_element(at_s=TARGET_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='target.4l3.b2_aper')




    return line

In [49]:
coll_R3 = os.path.expandvars('${HOME_TWOCRYST}/input_files/colldbs/flat_top.yaml')
line_file_R3 = os.path.expandvars("${HOME_TWOCRYST}/input_files/Run3_phase_scan/lines_ref/track_flat_top_b2.json")
particle_ref_R3= xp.Particles(p0c=6800e9, q0=1, mass0=xp.PROTON_MASS_EV)

In [9]:
coll_HL = os.path.expandvars('${HOME_TWOCRYST}/input_files/colldbs/HL_tight_twocryst.yaml')
line_file_HL = os.path.expandvars("${HOME_TWOCRYST}/input_files/HL_phase_scan/b4_sequence_patched_phadv_150.json")
particle_ref_HL= xp.Particles(p0c=7000e9, q0=1, mass0=xp.PROTON_MASS_EV)

In [50]:
line_R3 = load_line(line_file_R3, coll_R3, particle_ref_R3)
line_HL = load_line(line_file_HL, coll_HL, particle_ref_HL) 

Loading line from dict:   0%|          | 0/103397 [00:00<?, ?it/s]

Done loading line from dict.           


Loading line from dict:   0%|          | 0/151124 [00:00<?, ?it/s]

Done loading line from dict.           


In [11]:
tw_HL = line_HL.twiss(method='4d').to_pandas()
tw_R3 = line_R3.twiss(method='4d').to_pandas()

Found suitable prebuilt kernel `default_only_xtrack`.
Found suitable prebuilt kernel `only_xtrack_frozen_energy`.
Found suitable prebuilt kernel `default_only_xtrack`.
Found suitable prebuilt kernel `only_xtrack_frozen_energy`.


In [12]:
def calculate_delta_phase_adv(tw):
    return (float(tw[tw.name == TCCS_name]['muy'])% 1* 2*np.pi - float(tw[tw.name == TCP_name]['muy'])% 1* 2*np.pi)*180/np.pi

In [13]:
calculate_delta_phase_adv(tw_R3)

173.96466234204726

In [14]:
calculate_delta_phase_adv(tw_HL)

150.46979031963522

In [19]:
with open('../HL_phase_scan/knobs_db/knobs_159.pkl', 'rb') as f:
    knobs = pickle.load(f)

In [20]:
knobs

{'kqtf.b2': 0.001208677013518323,
 'kqtd.b2': 0.00038331555223907116,
 'ksf.b2': 0.013659436695399586,
 'ksd.b2': -0.027437069853417304,
 'kqt13.l2b2': 0.0004514499114322108,
 'kqt12.l2b2': -0.003255459636063764,
 'kqtl11.l2b2': 0.003621832707128442,
 'kq10.l2b2': 0.007038061942052969,
 'kq9.l2b2': -0.006933724300000922,
 'kq8.l2b2': 0.007049750740201287,
 'kq7.l2b2': -0.007838443884785605,
 'kq6.l2b2': 0.0043085171149305265,
 'kq5.l2b2': -0.003279698116036019,
 'kq4.l2b2': 0.0019927108461440654,
 'kqx.l2': 0.008755083323428548,
 'kq4.r2b2': -0.003630665367308744,
 'kq5.r2b2': 0.0048330772584889384,
 'kq6.r2b2': -0.003938045240378545,
 'kq7.r2b2': 0.00749567152642893,
 'kq8.r2b2': -0.0056231039407293225,
 'kq9.r2b2': 0.0067684978060658105,
 'kq10.r2b2': -0.005326779941068593,
 'kqtl11.r2b2': 0.0004494694925349877,
 'kqt12.r2b2': -0.0017493728475043034,
 'kqt13.r2b2': 0.004448848638839975,
 'kqt13.l4b2': -0.003479263576853937,
 'kqt12.l4b2': 0.0023715956762976787,
 'kqtl11.l4b2': -3.700

#  vars['kq8.l2b2']._get_value()
   vars['kq8.l2b2'] = 0.0070462830327643845

#  vars['kq8.l2b2']._expr is None

#  vars['kq8.l2b2']._find_dependant_targets()
   element_refs['mqml.8l2.b2..4'].knl[1]
   element_refs['mqml.8l2.b2..3'].knl[1]
   element_refs['mqml.8l2.b2..2'].knl[1]
   element_refs['mqml.8l2.b2..1'].knl[1]
   ... set _info(limit=None) to get all lines



In [33]:
line_HL.vv['kq8.l2b2']

0.0070462830327643845

In [ ]:
 strDictIR2 = {
        'kqt13.l2b2'  :{'step': 1e-6, 'limits':  [-qtlimit5,qtlimit5],},
        'kqt12.l2b2'  :{'step': 1e-6, 'limits':  [-qtlimit5,qtlimit5],},
        'kqtl11.l2b2' :{'step': 1e-6, 'limits':  [-qtlimit4,qtlimit4],},
        'kq10.l2b2'   :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq9.l2b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq8.l2b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq7.l2b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq6.l2b2'    :{'step': 1e-6, 'limits':  [-qtlimit2,qtlimit2],},
        'kq5.l2b2'    :{'step': 1e-6, 'limits':  [-qtlimit2,qtlimit2],},
        'kq4.l2b2'    :{'step': 1e-6, 'limits':  [-qtlimit2,qtlimit2],},
        'kqx.l2'      :{'step': 1e-6, 'limits':  [-qtlimitx28,qtlimitx28],},
        'kq4.r2b2'    :{'step': 1e-6, 'limits':  [-qtlimit2,qtlimit2],},
        'kq5.r2b2'    :{'step': 1e-6, 'limits':  [-qtlimit2,qtlimit2],},
        'kq6.r2b2'    :{'step': 1e-6, 'limits':  [-qtlimit2,qtlimit2],},
        'kq7.r2b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq8.r2b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq9.r2b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq10.r2b2'   :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kqtl11.r2b2' :{'step': 1e-6, 'limits':  [-qtlimit4,qtlimit4],},
        'kqt12.r2b2'  :{'step': 1e-6, 'limits':  [-qtlimit5,qtlimit5],},
        'kqt13.r2b2'  :{'step': 1e-6, 'limits':  [-qtlimit5,qtlimit5],},
    }

    strDictIR4 = {
        'kqt13.l4b2'  :{'step': 1e-6, 'limits':  [-qtlimit5,qtlimit5],},
        'kqt12.l4b2'  :{'step': 1e-6, 'limits':  [-qtlimit5,qtlimit5],},
        'kqtl11.l4b2' :{'step': 1e-6, 'limits':  [-qtlimit4,qtlimit4],},
        'kq10.l4b2'   :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq9.l4b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq8.l4b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq7.l4b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq6.l4b2'    :{'step': 1e-6, 'limits':  [-qtlimit2,qtlimit2],},
        'kq5.l4b2'    :{'step': 1e-6, 'limits':  [-qtlimit2,qtlimit2],},
        'kq5.r4b2'    :{'step': 1e-6, 'limits':  [-qtlimit2,qtlimit2],},
        'kq6.r4b2'    :{'step': 1e-6, 'limits':  [-qtlimit2,qtlimit2],},
        'kq7.r4b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq8.r4b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq9.r4b2'    :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kq10.r4b2'   :{'step': 1e-6, 'limits':  [-qtlimit3,qtlimit3],},
        'kqtl11.r4b2' :{'step': 1e-6, 'limits':  [-qtlimit4,qtlimit4],},
        'kqt12.r4b2'  :{'step': 1e-6, 'limits':  [-qtlimit5,qtlimit5],},
        'kqt13.r4b2'  :{'step': 1e-6, 'limits':  [-qtlimit5,qtlimit5],},
    }       

In [38]:
line_HLref = load_line( os.path.expandvars("${HOME_TWOCRYST}/input_files/HL_phase_scan/lines_ref/b4_sequence_noaper.json"), coll_HL, particle_ref_HL) 

Loading line from dict:   0%|          | 0/61371 [00:00<?, ?it/s]

Done loading line from dict.           


In [52]:
kn = 'kq8.l2b2'
line_HL.vars[kn]._info()

#  vars['kq8.l2b2']._get_value()
   vars['kq8.l2b2'] = 0.0070462830327643845

#  vars['kq8.l2b2']._expr is None

#  vars['kq8.l2b2']._find_dependant_targets()
   element_refs['mqml.8l2.b2..4'].knl[1]
   element_refs['mqml.8l2.b2..3'].knl[1]
   element_refs['mqml.8l2.b2..2'].knl[1]
   element_refs['mqml.8l2.b2..1'].knl[1]
   ... set _info(limit=None) to get all lines



In [53]:
line_HLref.vars[kn]._info()

#  vars['kq8.l2b2']._get_value()
   vars['kq8.l2b2'] = 0.00704606315897909

#  vars['kq8.l2b2']._expr is None

#  vars['kq8.l2b2']._find_dependant_targets()
   element_refs['mqml.8l2.b2..4'].knl[1]
   element_refs['mqml.8l2.b2..3'].knl[1]
   element_refs['mqml.8l2.b2..2'].knl[1]
   element_refs['mqml.8l2.b2..1'].knl[1]
   ... set _info(limit=None) to get all lines



In [54]:
line_R3.vars[kn]._info()

#  vars['kq8.l2b2']._get_value()
   vars['kq8.l2b2'] = 0.00601522183278

#  vars['kq8.l2b2']._expr is None

#  vars['kq8.l2b2']._find_dependant_targets()
   element_refs['mqml.8l2.b2..16'].knl[1]
   element_refs['mqml.8l2.b2..15'].knl[1]
   element_refs['mqml.8l2.b2..14'].knl[1]
   element_refs['mqml.8l2.b2..13'].knl[1]
   ... set _info(limit=None) to get all lines



In [51]:
line_R3.vars.keys()

['t_turn_s',
 '__vary_default',
 'version',
 'pi',
 'twopi',
 'degrad',
 'raddeg',
 'e',
 'amu0',
 'emass',
 'mumass',
 'nmass',
 'umass',
 'pmass',
 'clight',
 'qelect',
 'hbar',
 'erad',
 'prad',
 'twiss_tol',
 'l.tan',
 'l.tanal',
 'l.tanar',
 'l.tanc',
 'l.tct',
 'l.tcth',
 'l.tctva',
 'l.mbas2',
 'l.mbaw',
 'l.mbcs2',
 'l.mbls2',
 'l.mblw',
 'l.mbwmd',
 'l.mbxwh',
 'l.mbxws',
 'l.mbxwt',
 'mylhcbeam',
 'rematchir7',
 'createoutput',
 'make_thin',
 'lhclength',
 'sep_arc',
 'sep_ir3',
 'sep_ir4',
 'sep_ir7',
 'ip1ofs.b1',
 'ip1ofs.b2',
 'ip2ofs.b1',
 'ip2ofs.b2',
 'ip3ofs.b1',
 'ip3ofs.b2',
 'ip4ofs.b1',
 'ip4ofs.b2',
 'ip5ofs.b1',
 'ip5ofs.b2',
 'ip6ofs.b1',
 'ip6ofs.b2',
 'ip7ofs.b1',
 'ip7ofs.b2',
 'ip8ofs.b1',
 'ip8ofs.b2',
 'dsep1',
 'dsep2',
 'dsep3',
 'dsep4',
 'dsep5',
 'dsep7',
 'dsep8',
 'aip1',
 'aip2',
 'aip3',
 'aip4',
 'aip5',
 'aip7',
 'aip8',
 'pip1',
 'pip1.l1',
 'pip2',
 'pip3',
 'pip4',
 'pip5',
 'pip6',
 'pip7',
 'pip8',
 'hrf200',
 'hrf400',
 'a.mb',
 'ab.a12',

In [46]:
line_HLref.vars.keys()

['t_turn_s',
 '__vary_default',
 'version',
 'pi',
 'twopi',
 'degrad',
 'raddeg',
 'e',
 'amu0',
 'emass',
 'mumass',
 'nmass',
 'umass',
 'pmass',
 'clight',
 'qelect',
 'hbar',
 'erad',
 'prad',
 'none',
 'twiss_tol',
 'lhclength',
 'sep_arc',
 'sep_ir3',
 'sep_ir4',
 'sep_ir7',
 'ip1ofs.b1',
 'ip1ofs.b2',
 'ip2ofs.b1',
 'ip2ofs.b2',
 'ip3ofs.b1',
 'ip3ofs.b2',
 'ip4ofs.b1',
 'ip4ofs.b2',
 'ip5ofs.b1',
 'ip5ofs.b2',
 'ip6ofs.b1',
 'ip6ofs.b2',
 'ip7ofs.b1',
 'ip7ofs.b2',
 'ip8ofs.b1',
 'ip8ofs.b2',
 'dsep1',
 'dsep2',
 'dsep3',
 'dsep4',
 'dsep5',
 'dsep7',
 'dsep8',
 'aip1',
 'aip2',
 'aip3',
 'aip4',
 'aip5',
 'aip7',
 'aip8',
 'pip1',
 'pip1.l1',
 'pip2',
 'pip3',
 'pip4',
 'pip5',
 'pip6',
 'pip7',
 'pip8',
 'hrf200',
 'hrf400',
 'a.mb',
 'ab.a12',
 'ab.a23',
 'ab.a34',
 'ab.a45',
 'ab.a56',
 'ab.a67',
 'ab.a78',
 'ab.a81',
 'l.acsca',
 'l.acsph001',
 'l.acsph002',
 'l.adtkh',
 'l.adtkv',
 'l.apwl',
 'l.bbcwe',
 'l.bbcwi',
 'l.bctdc',
 'l.bctfr',
 'l.bgcab',
 'l.bgvca001',
 'l.b